### Notes

For Span obj:
+ build off author and publisher names
+ text
+ start char end char
+ label (author or publisher name)
+ group (make sure authors and pubs don't overlap)
+ as a group (this group's closeness to KB ids)
    + dictionary of id and then its score
    
For ReviewObj:
+ all author spans attribute
+ all pub spans attribute

Also refer to spaCy spans -> https://spacy.io/api/span
+ doc (doc object)
+ start (int)
+ end (int)
+ label (int/unicode)
+ kb_id (int/unicode)
+ vector (numpy.ndarray[ndim=1, dtype='float32'])

In [1]:
import sys
sys.path.append('../')

In [2]:
from application.name_obj_classes import PubName, PersonName
from application.review_obj_class import ReviewObj
import pandas as pd
import os
%pprint

Pretty printing has been turned OFF


In [25]:
from nltk.metrics import edit_distance

In [3]:
# loading in files
directory = "../../aps_reviews_50/aps_reviews/"
filenames = os.listdir(directory)
txts = []
for file in filenames:
    with open(directory + file) as f:
        txts.append(f.read())

In [4]:
review_list = []
for filename, txt in zip([x.split('.')[0] for x in filenames], txts):
    review_list.append(ReviewObj(filename, txt))

Span object will contain:
+ Review ID (parent doc)
+ Start char (int)
+ End char (int)
+ Label (person/publisher)
+ Group (int)
+ Name variations
+ VIAF but not yet

In [5]:
# review id
review_list[0].person_names[0].review_id

'136726613'

In [6]:
# start and end char
review_list[0].person_names[0].review_loc_chars

(95, 120)

In [7]:
# label
review_list[0].person_names[0].name_type

'person'

In [8]:
review_list[0].person_names[0].getNameVariants()

['c dudley warner', 'charles warner', 'charles dudley warner', 'warner', 'c d warner', 'charles d warner', 'c warner']

In [84]:
class NameSpan():
    def __init__(self, NameObj):
        self.name = NameObj
        self.review_id = NameObj.review_id
        self.span = NameObj.review_loc_chars
        self.start_char = NameObj.review_loc_chars[0]
        self.end_char = NameObj.review_loc_chars[1]
        self.label = NameObj.name_type
        self.name_id = int(span_test.review_id + str(self.start_char))
        self.group = -1

In [85]:
span_test = NameSpan(review_list[0].person_names[0])

In [86]:
span_test.review_id

'136726613'

In [87]:
span_test.name_id

13672661395

In [22]:
span_test.name

Mr. Charles Dudley Warner

In [21]:
span_test.name.last_name

'warner'

In [88]:
spanlist = [NameSpan(x) for x in review_list[0].person_names]

In [89]:
spanlist

[<__main__.NameSpan object at 0x1a2d379d90>, <__main__.NameSpan object at 0x1a2d379f50>, <__main__.NameSpan object at 0x1a2d379ad0>, <__main__.NameSpan object at 0x1a2d379650>, <__main__.NameSpan object at 0x1a2d379690>, <__main__.NameSpan object at 0x1a2d379610>, <__main__.NameSpan object at 0x1a2d379550>, <__main__.NameSpan object at 0x1a2d379fd0>, <__main__.NameSpan object at 0x1a2d3790d0>, <__main__.NameSpan object at 0x1a2d3797d0>, <__main__.NameSpan object at 0x1a2d379c90>, <__main__.NameSpan object at 0x1a2d379410>, <__main__.NameSpan object at 0x1a2d379e90>, <__main__.NameSpan object at 0x1a2d379d50>, <__main__.NameSpan object at 0x1a2d379090>, <__main__.NameSpan object at 0x1a2d379450>, <__main__.NameSpan object at 0x1a2d379a50>]

In [90]:
names = sorted([(span.name, span.name_id) for span in spanlist], key=lambda x: len(x[0].last_name))

In [91]:
names

[(Dr. Herbert B. Adam, 1367266134010), (Dr. Geor6 Ebers, 1367266131348), (President Andrew D. White, 1367266134149), (Mr. Charles Dudley Warner, 13672661395), (Mr. Vedder, 136726613190), (Professor Thorold Rogers, 136726613395), (Dr. Parker, 1367266135429), (Dr. William, 136726613837), (Mr. Cushing, 1367266133707), (Professor E. Johnsou, 1367266135306), (Dr. Malaren, 1367266135416), (Mrs. Browning, 136726613660), (Mr. Julius Chambers, 1367266131766), (Mr. Brander Matthews, 1367266134958), (Mr. Swinburne, 136726613819), (Mr. Julian Hawthorne, 1367266131620), (Dr. Joseph Parktercontinueshiseipositionuin, 1367266135237)]

In [55]:
names[0].last_name

'adam'

In [105]:
group_dict = {}
used_ids = []

In [106]:
for e, (name, name_id) in enumerate(names):
    if (name_id not in used_ids):
        id_holder = [name_id]
        for name2, name_id2 in names:
            if (e < (len(name) - 1)) and (name_id!=name_id2):
                if (edit_distance(name.last_name, name2.last_name[:len(name.last_name)+1]) < 2) and (name_id2 not in used_ids):
                    if (name.first_initial==name2.first_initial and name.middle_initial==name2.middle_initial or name.title==name2.title) or (name.first_initial==name2.first_initial or name.middle_initial==name2.middle_initial and name.title==name2.title):
                        id_holder.append(name_id2)
                        used_ids.append(name_id2)
        used_ids.append(name_id)
        for x in id_holder:
            group_dict[x] = e

In [107]:
group_dict

{1367266134010: 0, 1367266131348: 1, 1367266134149: 2, 13672661395: 3, 136726613190: 4, 136726613395: 5, 1367266135429: 6, 1367266135237: 6, 136726613837: 7, 1367266133707: 8, 1367266135306: 9, 1367266135416: 10, 136726613660: 11, 1367266131766: 12, 1367266134958: 13, 136726613819: 14, 1367266131620: 15}

In [111]:
for span in spanlist:
    span.group = group_dict[span.name_id]

In [112]:
for span in spanlist:
    print(span.name, span.group)

Mr. Charles Dudley Warner 3
Mr. Vedder 4
Professor Thorold Rogers 5
Mrs. Browning 11
Mr. Swinburne 14
Dr. William 7
Dr. Geor6 Ebers 1
Mr. Julian Hawthorne 15
Mr. Julius Chambers 12
Mr. Cushing 8
Dr. Herbert B. Adam 0
President Andrew D. White 2
Mr. Brander Matthews 13
Dr. Joseph Parktercontinueshiseipositionuin 6
Professor E. Johnsou 9
Dr. Malaren 10
Dr. Parker 6


In [114]:
def group_people(spanlist):
    
    names = sorted([(span.name, span.name_id) for span in spanlist], key=lambda x: len(x[0].last_name))
    group_dict = {}
    used_ids = []
    for e, (name, name_id) in enumerate(names):
        if (name_id not in used_ids):
            id_holder = [name_id]
            for name2, name_id2 in names:
                if (e < (len(name) - 1)) and (name_id!=name_id2):
                    if (edit_distance(name.last_name, name2.last_name[:len(name.last_name)+1]) < 2) and (name_id2 not in used_ids):
                        if (name.first_initial==name2.first_initial and name.middle_initial==name2.middle_initial or name.title==name2.title) or (name.first_initial==name2.first_initial or name.middle_initial==name2.middle_initial and name.title==name2.title):
                            id_holder.append(name_id2)
                            used_ids.append(name_id2)
            used_ids.append(name_id)
            for x in id_holder:
                group_dict[x] = e
    
    for span in spanlist:
        span.group = group_dict[span.name_id]
    
    return spanlist

In [115]:
spanlist = [NameSpan(x) for x in review_list[0].person_names]

In [116]:
spanlist = group_people(spanlist)

In [117]:
for span in spanlist:
    print(span.name, span.group)

Mr. Charles Dudley Warner 3
Mr. Vedder 4
Professor Thorold Rogers 5
Mrs. Browning 11
Mr. Swinburne 14
Dr. William 7
Dr. Geor6 Ebers 1
Mr. Julian Hawthorne 15
Mr. Julius Chambers 12
Mr. Cushing 8
Dr. Herbert B. Adam 0
President Andrew D. White 2
Mr. Brander Matthews 13
Dr. Joseph Parktercontinueshiseipositionuin 6
Professor E. Johnsou 9
Dr. Malaren 10
Dr. Parker 6


In [ ]:
class NameSpanGenerator():
    def _get_group:
        
    def _get_group_person